In [2]:
import os
import logging
import pickle
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases, LdaModel
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd

class TopicModeler:
    def __init__(self, input_path):
        self.input_path = input_path
        self.docs = []
        self.filtered_docs = []
        self.dictionary = None
        self.corpus = None
        self.model = None
        self.coherence_scores = []
        self.tokenizer = RegexpTokenizer(r'\w+')
        self.lemmatizer = WordNetLemmatizer()
        logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    def load_documents(self):
        for filename in os.listdir(self.input_path):
            file_path = os.path.join(self.input_path, filename)
            if os.path.isfile(file_path) and filename.endswith('.txt'):
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read().strip()
                    if isinstance(content, str):
                        self.docs.append(content)
        print(f"Loaded {len(self.docs)} documents")
        if self.docs:
            print(f"Sample document: {self.docs[0][:100]}...")

    def preprocess_documents(self):
        self.docs = [[token.lower() for token in self.tokenizer.tokenize(doc)] for doc in self.docs if isinstance(doc, str)]
        self.docs = [[token for token in doc if not token.isnumeric() and len(token) > 1] for doc in self.docs]
        stop_words = set(stopwords.words('english'))
        self.docs = [[word for word in doc if word not in stop_words] for doc in self.docs]
        self.docs = [[self.lemmatizer.lemmatize(token) for token in doc] for doc in self.docs]
        print(f"After preprocessing: {len(self.docs)} documents")
        if self.docs:
            print(f"Sample preprocessed document: {self.docs[0][:10]}...")

    def add_bigrams(self):
        bigram = Phrases(self.docs, min_count=5, threshold=100)
        for idx in range(len(self.docs)):
            for token in bigram[self.docs[idx]]:
                if '_' in token:
                    self.docs[idx].append(token)
        print(f"Added bigrams. Sample document: {self.docs[0][:15]}...")

    def filter_with_tfidf(self, top_n=200, bottom_n=50):
        texts = [' '.join(doc) for doc in self.docs]
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
        feature_names = tfidf_vectorizer.get_feature_names_out()
        mean_tfidf_scores = np.asarray(tfidf_matrix.mean(axis=0)).flatten()

      
        top_indices = mean_tfidf_scores.argsort()[-top_n:]
        bottom_indices = mean_tfidf_scores.argsort()[:bottom_n]
        
       
        words_to_remove = set(feature_names[i] for i in top_indices)
        
        self.filtered_docs = [[word for word in doc if word not in words_to_remove] for doc in self.docs]
        print(f"Filtered documents using TF-IDF. Sample filtered document: {self.filtered_docs[0][:10]}...")


    def create_dictionary_and_corpus(self):
        self.dictionary = Dictionary(self.filtered_docs)
        original_size = len(self.dictionary)
        self.dictionary.filter_extremes(no_below=2, no_above=0.9)
        self.corpus = [self.dictionary.doc2bow(doc) for doc in self.filtered_docs]
        print(f"Dictionary size: {len(self.dictionary)} (reduced from {original_size})")
        print(f"Corpus size: {len(self.corpus)}")
        if self.corpus:
            print(f"Sample corpus entry: {self.corpus[0][:10]}")

    def train_lda_model(self, num_topics=6, chunksize=2000, passes=20, iterations=400, alpha='auto', eta='auto'):
        if not self.corpus or not self.dictionary:
            raise ValueError("Corpus or dictionary is empty. Check your preprocessing steps.")
        
        print(f"Training LDA model with num_topics={num_topics}, chunksize={chunksize}, passes={passes}, iterations={iterations}, alpha={alpha}, eta={eta}")
        
        self.model = LdaModel(
            corpus=self.corpus,
            id2word=self.dictionary,
            chunksize=chunksize,
            alpha=alpha,
            eta=eta,
            iterations=iterations,
            num_topics=num_topics,
            passes=passes,
            eval_every=None,
            random_state=42
        )
        print("LDA model trained successfully")

    def print_model_info(self):
        print('Number of unique tokens: %d' % len(self.dictionary))
        print('Number of documents: %d' % len(self.corpus))
        top_topics = self.model.top_topics(self.corpus)
        avg_topic_coherence = sum([t[1] for t in top_topics]) / self.model.num_topics
        print('Average topic coherence: %.4f.' % avg_topic_coherence)
        print("\nTop topics:")
        pprint(self.model.print_topics())
        print('\nPerplexity: ', self.model.log_perplexity(self.corpus))
        coherence_model_lda = CoherenceModel(model=self.model, texts=self.filtered_docs, dictionary=self.dictionary, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        print('\nCoherence Score: ', coherence_lda)
        return coherence_lda

    def visualize_topics(self):
        vis = pyLDAvis.gensim.prepare(self.model, self.corpus, self.dictionary)
        return vis

    def analyze_documents(self):
        for i, doc in enumerate(self.filtered_docs[:5]):
            bow = self.dictionary.doc2bow(doc)
            doc_topics = self.model.get_document_topics(bow)
            print(f"\nDocument {i} topics:")
            pprint(doc_topics)
            best_topic = max(doc_topics, key=lambda x: x[1])
            print(f"Best topic: {best_topic[0]}")
            print(f"Top words in this topic:")
            pprint(self.model.show_topic(best_topic[0]))
            print(f"Original document: {' '.join(doc[:30])}...")
            print()

    def save_model(self, file_path):
        if self.model is None:
            raise ValueError("No model has been trained yet.")
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        self.model.save(file_path)
        print(f"Model saved to {file_path}")

    def load_model(self, file_path):
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"No model file found at {file_path}")
        self.model = LdaModel.load(file_path)
        print(f"Model loaded from {file_path}")

    def save_dictionary_and_corpus(self, dict_path, corpus_path):
        if self.dictionary is None or self.corpus is None:
            raise ValueError("Dictionary and corpus have not been created yet.")
        os.makedirs(os.path.dirname(dict_path), exist_ok=True)
        os.makedirs(os.path.dirname(corpus_path), exist_ok=True)
        self.dictionary.save(dict_path)
        with open(corpus_path, 'wb') as f:
            pickle.dump(self.corpus, f)
        print(f"Dictionary saved to {dict_path}")
        print(f"Corpus saved to {corpus_path}")

    def load_dictionary_and_corpus(self, dict_path, corpus_path):
        if not os.path.exists(dict_path) or not os.path.exists(corpus_path):
            raise FileNotFoundError(f"Dictionary or corpus file not found.")
        self.dictionary = Dictionary.load(dict_path)
        with open(corpus_path, 'rb') as f:
            self.corpus = pickle.load(f)
        print(f"Dictionary loaded from {dict_path}")
        print(f"Corpus loaded from {corpus_path}")

    def plot_coherence_scores(self, iteration):
        num_topics = [x[0] for x in self.coherence_scores]
        coherences = [x[1] for x in self.coherence_scores]
        
        plt.figure(figsize=(10, 6))
        plt.plot(num_topics, coherences, marker='o')
        plt.xlabel('Number of Topics')
        plt.ylabel('Coherence Score')
        plt.title('Coherence Score by Number of Topics')
        plt.grid(True)
        
        plt.xticks(num_topics)
        plt.ylim(0, max(coherences) * 1.1)
        
        plt.savefig(f'coherence_scores_iteration_{iteration}.png')
        plt.close()

    def get_word_frequencies(self):
        topics = self.model.get_topics()
        word_frequencies = np.sum(topics, axis=0)
        word_frequencies /= self.model.num_topics
        return {self.dictionary[i]: freq for i, freq in enumerate(word_frequencies)}

    def get_low_frequency_words(self, threshold=0.01):
        word_frequencies = self.get_word_frequencies()
        return {word: freq for word, freq in word_frequencies.items() if freq < threshold}

    def get_high_frequency_words(self, threshold=0.00016):
        word_frequencies = self.get_word_frequencies()
        high_freq_words = {word: freq for word, freq in word_frequencies.items() if freq > threshold}
        
        if not high_freq_words:
            print(f"No words found with a frequency higher than the threshold of {threshold}.")
        
        return high_freq_words


    def save_frequency_words(self, low_freq_file, high_freq_file, low_threshold=0.01, high_threshold=0.1):
        low_freq_words = self.get_low_frequency_words(low_threshold)
        high_freq_words = self.get_high_frequency_words(high_threshold)

        df_low = pd.DataFrame(list(low_freq_words.items()), columns=['Word', 'Frequency'])
        df_low = df_low.sort_values('Frequency', ascending=True)
        df_low.to_csv(low_freq_file, index=False)
        print(f"Low frequency words saved to {low_freq_file}")

        df_high = pd.DataFrame(list(high_freq_words.items()), columns=['Word', 'Frequency'])
        df_high = df_high.sort_values('Frequency', ascending=False)
        df_high.to_csv(high_freq_file, index=False)
        print(f"High frequency words saved to {high_freq_file}")

    def run_with_params(self, num_topics, chunksize, passes, iterations):
        try:
            self.load_documents()
            self.preprocess_documents()
            self.add_bigrams()
            self.filter_with_tfidf(top_n=200, bottom_n=50)
            self.create_dictionary_and_corpus()
            self.train_lda_model(num_topics=num_topics, chunksize=chunksize, passes=passes, iterations=iterations)
            model_dir = f"lda_models/topics_{num_topics}_passes_{passes}_iterations_{iterations}"
            self.save_model(os.path.join(model_dir, 'trained_model'))
            self.save_dictionary_and_corpus(os.path.join(model_dir, 'dictionary'), os.path.join(model_dir, 'corpus'))
            coherence_lda = self.print_model_info()
            self.coherence_scores.append((num_topics, coherence_lda))
            
            # Plot and save coherence score after each run
            self.plot_coherence_scores(len(self.coherence_scores))
            
            vis = self.visualize_topics()
            pyLDAvis.save_html(vis, os.path.join(model_dir, 'lda_visualization.html'))
            self.analyze_documents()
            
            return vis
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            raise

if __name__ == "__main__":
    modeler = TopicModeler('txt')

    param_combinations = [
        (6, 2000, 10, 200),
        (8, 2000, 10, 400),
        (10, 2000, 20, 200),
        (12, 2000, 20, 400),
        (14, 2000, 20, 400),
        (16, 2000, 20, 400)
    ]

    csv_folder = 'csv_output'
    os.makedirs(csv_folder, exist_ok=True)

    for num_topics, chunksize, passes, iterations in param_combinations:
        print(f"\nRunning model with parameters: num_topics={num_topics}, chunksize={chunksize}, passes={passes}, iterations={iterations}")
        modeler.run_with_params(num_topics, chunksize, passes, iterations)
    
    modeler.save_frequency_words(
        os.path.join(csv_folder, 'low_freq_words.csv'),
        os.path.join(csv_folder, 'high_freq_words.csv'),
        low_threshold=0.01,
    high_threshold=0.00016
    )

2024-08-06 18:31:30,914 : INFO : collecting all words and their counts
2024-08-06 18:31:30,914 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2024-08-06 18:31:30,935 : INFO : collected 23445 token types (unigram + bigrams) from a corpus of 44718 words and 440 sentences



Running model with parameters: num_topics=6, chunksize=2000, passes=10, iterations=200
Loaded 440 documents
Sample document: Pricing for interconnection related to the provision of number portability, as referred to in Articl...
After preprocessing: 440 documents
Sample preprocessed document: ['pricing', 'interconnection', 'related', 'provision', 'number', 'portability', 'referred', 'article', 'directive', 'ec']...


2024-08-06 18:31:30,936 : INFO : merged Phrases<23445 vocab, min_count=5, threshold=100, max_vocab_size=40000000>
2024-08-06 18:31:30,936 : INFO : Phrases lifecycle event {'msg': 'built Phrases<23445 vocab, min_count=5, threshold=100, max_vocab_size=40000000> in 0.02s', 'datetime': '2024-08-06T18:31:30.936710', 'gensim': '4.3.2', 'python': '3.11.9 (main, Jul 16 2024, 11:07:54) [Clang 14.0.0 (clang-1400.0.29.202)]', 'platform': 'macOS-12.5-arm64-arm-64bit', 'event': 'created'}
2024-08-06 18:31:31,010 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-08-06 18:31:31,021 : INFO : built Dictionary<3466 unique tokens: ['abstract', 'adoption', 'advance', 'appeal', 'body']...> from 440 documents (total 19293 corpus positions)
2024-08-06 18:31:31,022 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<3466 unique tokens: ['abstract', 'adoption', 'advance', 'appeal', 'body']...> from 440 documents (total 19293 corpus positions)", 'datetime': '2024-08-06T18:31:31.02232

Added bigrams. Sample document: ['pricing', 'interconnection', 'related', 'provision', 'number', 'portability', 'referred', 'article', 'directive', 'ec', 'european', 'parliament', 'council', 'march', 'universal']...
Filtered documents using TF-IDF. Sample filtered document: ['pricing', 'interconnection', 'related', 'number', 'portability', 'concern', 'traffic', 'number', 'ported', 'set']...
Dictionary size: 1835 (reduced from 3466)
Corpus size: 440
Sample corpus entry: [(0, 1), (1, 1), (2, 3), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]
Training LDA model with num_topics=6, chunksize=2000, passes=10, iterations=200, alpha=auto, eta=auto


2024-08-06 18:31:31,210 : INFO : optimized alpha [0.08766382, 0.0900195, 0.08876704, 0.09682601, 0.10300999, 0.09258881]
2024-08-06 18:31:31,211 : INFO : topic #0 (0.088): 0.007*"property" + 0.006*"emergency" + 0.005*"number" + 0.005*"call" + 0.005*"advertiser" + 0.005*"advertisement" + 0.005*"trade_mark" + 0.004*"single" + 0.004*"immovable" + 0.004*"data"
2024-08-06 18:31:31,212 : INFO : topic #2 (0.089): 0.006*"set" + 0.005*"established" + 0.005*"january" + 0.005*"consequence" + 0.004*"procedure" + 0.004*"ass" + 0.004*"operator" + 0.004*"ensure" + 0.004*"determine" + 0.004*"result"
2024-08-06 18:31:31,212 : INFO : topic #5 (0.093): 0.008*"currency" + 0.007*"foreign" + 0.007*"spain" + 0.006*"kingdom_spain" + 0.006*"device" + 0.006*"exchange" + 0.004*"production" + 0.004*"financial" + 0.004*"vehicle" + 0.004*"provide"
2024-08-06 18:31:31,213 : INFO : topic #3 (0.097): 0.008*"le" + 0.007*"et" + 0.007*"take" + 0.006*"la" + 0.005*"en" + 0.005*"reference" + 0.004*"nature" + 0.004*"conseque

...


Document 0 topics:
[(8, 0.9936895)]
Best topic: 8
Top words in this topic:
[('directory', 0.0147787165),
 ('data', 0.012977489),
 ('property', 0.01212701),
 ('provider', 0.010293842),
 ('making', 0.010158941),
 ('subscriber', 0.010024913),
 ('pre', 0.009290173),
 ('telephone', 0.00925566),
 ('number', 0.0075557875),
 ('operator', 0.0075398507)]
Original document: pricing interconnection related number portability concern traffic number ported set incurred mobile telephone operator implement request number porting adoption laying method used calculating fix advance abstract model maximum charged donor...


Document 1 topics:
[(9, 0.27961), (13, 0.70689225)]
Best topic: 13
Top words in this topic:
[('natural', 0.014889866),
 ('purchaser', 0.011159046),
 ('understood', 0.008210618),
 ('paid', 0.007855486),
 ('mortgage', 0.00719655),
 ('scheme', 0.0069272364),
 ('holiday', 0.006533294),
 ('natural_person', 0.006389067),
 ('travel', 0.006293065),
 ('manufacturer', 0.0061782184)]
Original d

2024-08-06 18:31:48,635 : INFO : using autotuned alpha, starting with [0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625]
2024-08-06 18:31:48,636 : INFO : using serial LDA version on this node
2024-08-06 18:31:48,638 : INFO : running online (multi-pass) LDA training, 16 topics, 20 passes over the supplied corpus of 440 documents, updating model once every 440 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2024-08-06 18:31:48,638 : INFO : PROGRESS: pass 0, at document #440/440
2024-08-06 18:31:48,811 : INFO : optimized alpha [0.052009016, 0.052649535, 0.051226173, 0.05046125, 0.053348713, 0.050358627, 0.05136916, 0.04964234, 0.05160689, 0.05274506, 0.04967682, 0.04969019, 0.051447958, 0.050722785, 0.050538506, 0.053171054]
2024-08-06 18:31:48,813 : INFO : topic #7 (0.050): 0.010*"charge" + 0.008*"authorisation" + 0.008*"licence" + 0.008*"device" + 0.0

LDA model trained successfully
Model saved to lda_models/topics_16_passes_20_iterations_400/trained_model
Dictionary saved to lda_models/topics_16_passes_20_iterations_400/dictionary
Corpus saved to lda_models/topics_16_passes_20_iterations_400/corpus
Number of unique tokens: 1835
Number of documents: 440
Average topic coherence: -9.6909.

Top topics:
[(0,
  '0.014*"call" + 0.014*"property" + 0.012*"emergency" + 0.010*"premise" + '
  '0.010*"number" + 0.008*"cancellation" + 0.008*"away" + 0.007*"available" + '
  '0.007*"caller" + 0.007*"telephone"'),
 (1,
  '0.013*"protected_designation" + 0.011*"television" + 0.009*"system" + '
  '0.009*"fact" + 0.008*"agricultural" + 0.008*"registration" + 0.008*"used" + '
  '0.008*"channel" + 0.007*"wine" + 0.007*"specification"'),
 (2,
  '0.007*"pre" + 0.007*"registration" + 0.007*"failure" + 0.007*"ass" + '
  '0.007*"brought" + 0.006*"following" + 0.006*"kg" + 0.006*"body" + '
  '0.006*"ensure" + 0.005*"appropriate"'),
 (3,
  '0.019*"le" + 0.018*"

2024-08-06 18:31:51,715 : INFO : 9 accumulators retrieved from output queue
2024-08-06 18:31:51,750 : INFO : accumulated word occurrence stats for 1824 virtual documents



Coherence Score:  0.42060175306907976

Document 0 topics:
[(8, 0.99313956)]
Best topic: 8
Top words in this topic:
[('directory', 0.016729774),
 ('data', 0.015686158),
 ('property', 0.013388031),
 ('provider', 0.011527475),
 ('subscriber', 0.01047814),
 ('telephone', 0.010249694),
 ('number', 0.009823517),
 ('individual', 0.008216942),
 ('territory', 0.008039163),
 ('finland', 0.0073567485)]
Original document: pricing interconnection related number portability concern traffic number ported set incurred mobile telephone operator implement request number porting adoption laying method used calculating fix advance abstract model maximum charged donor...


Document 1 topics:
[(13, 0.98400766)]
Best topic: 13
Top words in this topic:
[('natural', 0.019501213),
 ('natural_person', 0.01035301),
 ('fact', 0.00921383),
 ('profession', 0.00886794),
 ('brought', 0.008130042),
 ('acting', 0.008122334),
 ('substance', 0.008073993),
 ('also', 0.0071520647),
 ('defendant', 0.0069301594),
 ('company'